In [10]:
from sage.crypto.sbox import AES
S = SBox([12,5,6,11,9,0,10,13,3,14,15,8,4,7,1,2])
# S.differential_distribution_;
n = 8
M = 200
S = set([(16)*diff+S(x)^^S(x^^diff) for diff in range((16)) for x in range(16)])
[x for x in S]
F2n = set([16*x+y for x in range(16) for y in range(16)])

S_complement = F2n.difference(S)

(16, -16, 0, -24, 8, -8, 8, -16, 24, -8, -24, 16, 24, 8, 8, -16, 16, -8, -16, -16, -8, 16, -8, 24, -24, -32, -16, 16, 16, 24, -8, 24, -8, -8, 0, -8, -8, 8, 0, -8, 24, -8, -16, -24, 16, 0, -8, -16, -16, -8, 8, -8, -16, 8, 24, 8)

In [42]:
flag = 0
k = 5
p = MixedIntegerLinearProgram(solver = "GLPK")
b_i = p.new_variable(integer=True, nonnegative=False)
a_ij = p.new_variable(integer=True, nonnegative=False)
d_iy = p.new_variable(integer=True, nonnegative=True)
for i in range(k):
    p.set_max(b_i[i],0)
for index in range(len(S_complement)):
    for i in range(k):
        p.set_max(d_iy[i,index],1)
for x in S:
    x_binary = x.bits()
    if len(x_binary) != n:
        for i in range(n-len(x_binary)):
            x_binary.append(0) 
    for i in range(k):
        p.add_constraint(-1*b_i[i] + sum(a_ij[i,j]*x_binary[j] for j in range(n)) >= 0)

index = 0
for y in S_complement:
    y_binary = y.bits()
    if len(y_binary) != n:
        for i in range(n-len(y_binary)):
            y_binary.append(0)
    p.add_constraint(sum([d_iy[i,index] for i in range(k)]) >= 1)
    for i in range(k):
        p.add_constraint(-1*b_i[i] + sum(a_ij[i,j]*y_binary[j] for j in range(n)) - M*(1-d_iy[i,index]) <= -1)
#         p.add_constraint(-1*b_i[i] + sum(a_ij[i,j]*y_binary[j] for j in range(n)) - M*(-d_iy[i,index]) >= 0)
    index = index + 1
#     p.set_objective(sum([ba_idx[i] for i in [n+1..n+len(Byr)]]))
# p.show()
p.solve()

b_value = p.get_values(b_i)
a_value = p.get_values(a_ij)
# print(a_value, type(a_value))
for i in range(k):
    print(b_value[i], end=" ")
    for j in range(n):
        print(a_value[i,j], end=" ")
    print("\n")
# print("b_i:",p.get_values(b_i))
# print("\n")
# print("a_ij:",p.get_values(a_ij))
# print("\n")
# print("d_iy:",p.get_values(d_iy))

KeyboardInterrupt: 

TypeError: unsupported operand type(s) for -: 'sage.numerical.linear_functions.LinearFunction' and 'sage.numerical.mip.MIPVariable'

{0: 0.0, 1: 2.0, 2: 2.0}